# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
from pyspark.sql.functions import col, when, sum as spark_sum, collect_list, udf, struct, desc
from pyspark.sql.window import Window
import json
from pyspark.sql.types import StringType

In [3]:
spark = SparkSession.builder \
    .appName("UserRoutesAnalysis") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-25 14:01:23,679 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Import data

In [4]:
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("session_id", IntegerType(), True),
    StructField("event_type", StringType(), True),
    StructField("event_page", StringType(), True),
    StructField("timestamp", LongType(), True)
])

cs_df = spark.read.csv(
    "hdfs:/data/clickstream.csv",
    sep="\t",
    header=False,
    schema=schema
)

Errors works

In [5]:
cs_df = cs_df.withColumn(
    "is_err",
    when(col("event_type").rlike("(?i)error"), 1).otherwise(0)
)

win_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

cs_df = cs_df.withColumn(
    "err_flag",
    spark_sum("is_err").over(win_spec)
)

valid_cs_df = cs_df.filter(col("err_flag") == 0)
valid_cs_df = valid_cs_df.filter(col("event_page").isNotNull())
valid_cs_df = valid_cs_df.filter(col("user_id") > 0)
valid_cs_df.cache()

vcount = valid_cs_df.count()
print(f"Number of ok-rows: {vcount}")

Number of ok-rows: 698348


Grouping

In [6]:
ev_df = valid_cs_df.select("user_id", "session_id", "timestamp", "event_page") \
    .withColumn("event", struct(col("timestamp"), col("event_page")))

rt_df = ev_df.groupBy("user_id", "session_id") \
    .agg(collect_list("event").alias("events"))

Route building

In [7]:
def build_rt(events):
    events = sorted(events, key=lambda x: x['timestamp'])
    pages = []
    prev_page = None
    for event in events:
        page = event['event_page']
        if page != prev_page:
            pages.append(page)
            prev_page = page
    return "-".join(pages)

build_rt_udf = udf(build_rt, StringType())

Top 10 routes for DataFrame API

In [8]:
rt_df = rt_df.withColumn("route", build_rt_udf("events"))

rt_cnt_df = rt_df.groupBy("route").count()

top10_rt_df = rt_cnt_df.orderBy(desc("count")).limit(10)

print("Top 10 routes for DataFrame API")
top10_rt_df.show(truncate=False)

Top 10 routes for DataFrame API


+-------------------+-----+
|route              |count|
+-------------------+-----+
|main               |8183 |
|main-archive       |1113 |
|main-rabota        |1047 |
|main-internet      |897  |
|main-bonus         |870  |
|main-news          |769  |
|main-tariffs       |677  |
|main-online        |587  |
|main-vklad         |517  |
|main-rabota-archive|170  |
+-------------------+-----+



Top 10 routes for Spark SQL

In [9]:
valid_cs_df.createOrReplaceTempView("valid_clickstream")
spark.udf.register("build_rt_sql", build_rt, StringType())

top10_rt_sql_df = spark.sql("""
SELECT
    route,
    COUNT(*) as count
FROM (
    SELECT
        user_id,
        session_id,
        build_rt_sql(COLLECT_LIST(NAMED_STRUCT('timestamp', timestamp, 'event_page', event_page))) as route
    FROM valid_clickstream
    GROUP BY user_id, session_id
) t
GROUP BY route
ORDER BY count DESC
LIMIT 10
""")

print("Top 10 routes for Spark SQL")
top10_rt_sql_df.show(truncate=False)

Top 10 routes for Spark SQL


+-------------------+-----+
|route              |count|
+-------------------+-----+
|main               |8183 |
|main-archive       |1113 |
|main-rabota        |1047 |
|main-internet      |897  |
|main-bonus         |870  |
|main-news          |769  |
|main-tariffs       |677  |
|main-online        |587  |
|main-vklad         |517  |
|main-rabota-archive|170  |
+-------------------+-----+



Top 10 routes for RDD API

In [10]:
routes_rdd = valid_cs_df.select("user_id", "session_id", "timestamp", "event_page") \
    .rdd.map(lambda row: ((row['user_id'], row['session_id']), (row['timestamp'], row['event_page'])))

grouped_rdd = routes_rdd.groupByKey()

def build_rt_rdd(events):
    events = sorted(events, key=lambda x: x[0])
    pages = []
    prev_page = None
    for ts, page in events:
        if page != prev_page:
            pages.append(page)
            prev_page = page
    return "-".join(pages)

routes_rdd = grouped_rdd.mapValues(build_rt_rdd)

rt_cnt_rdd = routes_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)

top10_rt_rdd = rt_cnt_rdd.takeOrdered(10, key=lambda x: -x[1])

top10_rt_rdd_df = spark.createDataFrame(top10_rt_rdd, ["route", "count"])

print("Top 10 routes for RDD API")
top10_rt_rdd_df.orderBy(desc("count")).show(truncate=False)

Top 10 routes for RDD API


+-------------------+-----+
|route              |count|
+-------------------+-----+
|main               |8183 |
|main-archive       |1113 |
|main-rabota        |1047 |
|main-internet      |897  |
|main-bonus         |870  |
|main-news          |769  |
|main-tariffs       |677  |
|main-online        |587  |
|main-vklad         |517  |
|main-rabota-archive|170  |
+-------------------+-----+



Comparing results

In [12]:
df_routes = top10_rt_df.collect()
sql_routes = top10_rt_sql_df.collect()
rdd_routes = top10_rt_rdd_df.collect()

def compare_routes(routes1, routes2, routes3):
    set1 = set((row['route'], row['count']) for row in routes1)
    set2 = set((row['route'], row['count']) for row in routes2)
    set3 = set((row['route'], row['count']) for row in routes3)
    return set1 == set2 == set3

are_equal = compare_routes(df_routes, sql_routes, rdd_routes)
print(f"Results match ?: {are_equal}")

result_dict = {row['route']: row['count'] for row in df_routes}

Results match ?: True


Creating json file

In [14]:
with open("result.json", "w", encoding='utf-8') as f:
    json.dump(result_dict, f, ensure_ascii=False, indent=4)

print("result.json done")

result.json done


In [15]:
!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/mlitvinov/w6/2

0.8999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Wrong main-vklad answer!
Correct main-rabota-archive answer!
